# Creating Your First Graph and Running It in a Session

In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2
print(f)

Tensor("add_1:0", shape=(), dtype=int32)


In [3]:
# 실행방법1
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


In [4]:
# 실행방법2
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)

42


In [5]:
# 초기화 방법 
init = tf.global_variables_initializer() # prepare an init node
with tf.Session() as sess:
    init.run() # 실제로 모든 변수가 초기화되는 시점 
    result = f.eval()
    print(result)

42


In [6]:
# with 블럭 없이 자동 초기화
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


# Managing Graphs

In [7]:
# 모든 노드는 그래프 객체가 있음
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [8]:
# 독립적인 그래프 객체 만들기
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())
tf.reset_default_graph() # 그래프 객체를 가끔 reset 해주는 것이 좋음

True
False


# Lifecycle of a Node Value

In [9]:
# 첫번째 w 에 의존하여 x, y, z 생성 
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3
with tf.Session() as sess:
    print(y.eval())# 결과를 출력하고 결과는 삭제됨 
    print(z.eval())#w, x 그래프를 한번 더 탐
    y_val, z_val = sess.run([y, z]) # w, x 그래프를 한번에 계산함 
    print(y_val)
    print(z_val) 
# 각각 다른 세션은 어떤 상태도 공유하지 않는데 
# 같은 graph 를 사용하더라도 공유되지 않는다.

10
15
10
15


# Linear Regression with TensorFlow

In [10]:
import numpy as np
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
m, n = housing.data.shape
print(m,n)
print(housing.data[:4])
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
# 하우징 데이터에 바이어스 1col 을 추가해서 만들어줌
print(housing_data_plus_bias.data[:4])

# tansorboard 에서 알아 볼수 있도록 이름여서 정의
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X") 
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)

# theta : 정규 방정식을 이용한 상관계수 (normal equation)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
with tf.Session() as sess:
    theta_value = theta.eval()
    print(theta_value)

20640 8
[[  8.32520000e+00   4.10000000e+01   6.98412698e+00   1.02380952e+00
    3.22000000e+02   2.55555556e+00   3.78800000e+01  -1.22230000e+02]
 [  8.30140000e+00   2.10000000e+01   6.23813708e+00   9.71880492e-01
    2.40100000e+03   2.10984183e+00   3.78600000e+01  -1.22220000e+02]
 [  7.25740000e+00   5.20000000e+01   8.28813559e+00   1.07344633e+00
    4.96000000e+02   2.80225989e+00   3.78500000e+01  -1.22240000e+02]
 [  5.64310000e+00   5.20000000e+01   5.81735160e+00   1.07305936e+00
    5.58000000e+02   2.54794521e+00   3.78500000e+01  -1.22250000e+02]]
[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


# Implementing Gradient Descent

In [11]:
# TensorFlow’s autodiff feature
# TensorFlow’s out-of-the-box optimizers
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

## Manually Computing the Gradients

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # 스케일러 평균 0 의 z score로 변경해줌 학습에 맞춰 데이터를 줄여줌 
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [13]:
print(scaled_housing_data_plus_bias.mean(axis=0))
print(scaled_housing_data_plus_bias.mean(axis=1))
print(scaled_housing_data_plus_bias.mean())
print(scaled_housing_data_plus_bias.shape)

[  1.00000000e+00   6.60969987e-17   5.50808322e-18   6.60969987e-17
  -1.06030602e-16  -1.10161664e-17   3.44255201e-18  -1.07958431e-15
  -8.52651283e-15]
[ 0.38915536  0.36424355  0.5116157  ..., -0.06612179 -0.06360587
  0.01359031]
0.111111111111
(20640, 9)


In [14]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

# random_uniform() : 정해진 shape 과 구간에 맞는 랜덤값을 가지는 노드 생성 
# numpy 의 rand 함수와 비슷 theta(W)
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m*tf.matmul(tf.transpose(X),error)

# assign() batch gradient descent 를 구연하여 theta 에 선언함 
# 기존과는 다른 type 혹은 shape이라면 텐서플로우는 오류
# shape을 바꿀 상황에 그 땐 assign을 사용 
# https://tfedohk.github.io/tensorflow/2017/01/07/tensorflow-datatype.html#gsc.tab=0
training_op = tf.assign(theta, theta-learning_rate * gradients)
print(theta is training_op)
# theta = theta-learning_rate * gradients

False


In [15]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()
# 순환문을 돌때 마다 mse 가 줄어 듬

Epoch 0 MSE = 9.34141
Epoch 100 MSE = 0.954206
Epoch 200 MSE = 0.71805
Epoch 300 MSE = 0.661502
Epoch 400 MSE = 0.623511
Epoch 500 MSE = 0.596159
Epoch 600 MSE = 0.576386
Epoch 700 MSE = 0.562083
Epoch 800 MSE = 0.551731
Epoch 900 MSE = 0.544236


## Using autodiff

In [16]:
# gradients = 2/m*tf.matmul(tf.transpose(X),error) 
# ↓
gradients = tf.gradients(mse, [theta])[0]
# tensorflow 에서 제공하고 있는 함수를 사용

## Using an Optimizer

In [17]:
# gradients = 2/m*tf.matmul(tf.transpose(X),error)
# training_op = tf.assign(theta, theta-learning_rate * gradients)
# ↓
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
# 모멘텀 최적화 방식 : loc, grobal 아다그리드 내려갈땐 더 빠르게 가속을 줌, loc minimum 에 빠졌을때 빠져나갈 수 있음 (관성이 존재함) 
# optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)

# Feeding Data to the Training Algorithm

## Placeholder nodes

In [18]:
import tensorflow as tf
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A+5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})
print(B_val_1)
print(B_val_2)

[[ 6.  7.  8.]]
[[  9.  10.  11.]
 [ 12.  13.  14.]]


## Mini-batch Gradient Descent

In [19]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [20]:
import numpy as np
import numpy.random as rnd
import os

# disk 에서 data 읽어 오기
def fetch_batch(epoch, batch_index, batch_size):
    rnd.seed(epoch * n_batches + batch_index)
    indices = rnd.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()
    
print("Best theta:")
print(best_theta)

Best theta:
[[ 2.07001591]
 [ 0.82045609]
 [ 0.1173173 ]
 [-0.22739051]
 [ 0.31134021]
 [ 0.00353193]
 [-0.01126994]
 [-0.91643935]
 [-0.87950081]]


# Saving and Restoring Models

In [21]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [22]:
# save 로 체크 포인트를 설정
init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            save_path = saver.save(sess, "C:/Users/jhee/Documents/StudyFolder/SlowML/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "C:/Users/jhee/Documents/StudyFolder/SlowML/tmp/my_model_final.ckpt")

print("Best theta:")
print(best_theta)

Epoch 0 MSE = 2.75443
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.57278
Epoch 300 MSE = 0.558501
Epoch 400 MSE = 0.549069
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.537379
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.531243
Epoch 900 MSE = 0.529371
Best theta:
[[  2.06855249e+00]
 [  7.74078071e-01]
 [  1.31192386e-01]
 [ -1.17845066e-01]
 [  1.64778143e-01]
 [  7.44078017e-04]
 [ -3.91945094e-02]
 [ -8.61356676e-01]
 [ -8.23479772e-01]]


In [23]:
# Resore 
with tf.Session() as sess:
    saver.restore(sess, "C:/Users/jhee/Documents/StudyFolder/SlowML/tmp/my_model_final.ckpt")
[...]

IndentationError: expected an indented block (<ipython-input-23-ca436751cc1c>, line 3)

In [24]:
# 더 많은 제어가 필요한 경우
# theta 변수에 weights 라는 이름으로 저장하거나 복원 할수 있음
saver = tf.train.Saver({"weights": theta})

# Visualizing the Graph and Training Curves Using TensorBoard


## inside jupyter

In [25]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [26]:
show_graph(tf.get_default_graph())

## Using TensorBoard

In [27]:
tf.reset_default_graph()

# timestamp 를 이용하여 log 디렉토리 구분
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

# tensorboard 에 그릴 mse 정보를 바이너리 파일로 만들어 그래프 노드를 생성
mse_summary = tf.summary.scalar('MSE', mse)
# 디렉토리에 저장할 file 노드 생성 (생성된 파일을 이벤트 파일이라함)
summary_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [28]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                # 10번 돌때 마다 mse_summary 노드를 업데이트 시킴 
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                summary_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

summary_writer.flush()
summary_writer.close()
print("Best theta:")
print(best_theta)

Best theta:
[[ 2.07001591]
 [ 0.82045609]
 [ 0.1173173 ]
 [-0.22739051]
 [ 0.31134021]
 [ 0.00353193]
 [-0.01126994]
 [-0.91643935]
 [-0.87950081]]


저장된 파일로 tensorboard 실행하기 
cmd 창에서<br/>
\> activate tensorflow<br/>
\> tensorboard --logdir tf_logs <br/>
http://localhost:6006/

# Name Scope

In [29]:
# >tensorboard --logdir logs : board에서 확인 하기 
# 좀 더 복잡한 구조를 단순화 시키기 위해서 Name Space 이용
tf.reset_default_graph()

now = datetime.utcnow().strftime("%Y%m%d%H%M%S") # utcnow 현재 시간이 나옴 
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

# error, mse 노드 정의를 name_scope 에 loss 이름 지정 
with tf.name_scope('loss') as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
    
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
summary_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                summary_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

summary_writer.flush()
summary_writer.close()
print("Best theta:")
print(best_theta)

Best theta:
[[ 2.07001591]
 [ 0.82045609]
 [ 0.1173173 ]
 [-0.22739051]
 [ 0.31134021]
 [ 0.00353193]
 [-0.01126994]
 [-0.91643935]
 [-0.87950081]]


In [30]:
print(error.op.name)
print(mse.op.name)

loss/sub
loss/mse


In [31]:
tf.reset_default_graph()

a1 = tf.Variable(0, name="a")      # name == "a"
a2 = tf.Variable(0, name="a")      # name == "a_1"

with tf.name_scope("param"):       # name == "param"
    a3 = tf.Variable(0, name="a")  # name == "param/a"

with tf.name_scope("param"):       # name == "param_1"
    a4 = tf.Variable(0, name="a")  # name == "param_1/a"

for node in (a1, a2, a3, a4):
    print(node.op.name)

a
a_1
param/a
param_1/a


# Modularity

In [32]:
tf.reset_default_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

linear1 = tf.add(tf.matmul(X, w1), b1, name="linear1")
linear2 = tf.add(tf.matmul(X, w2), b2, name="linear2")

relu1 = tf.maximum(linear1, 0, name="relu1")
relu2 = tf.maximum(linear1, 0, name="relu2")  # Oops, cut&paste error! Did you spot it?

output = tf.add_n([relu1, relu2], name="output")

In [33]:
# ReLU 함수를 이용하여 반복 작업을 줄임

tf.reset_default_graph()

def relu(X):
    w_shape = int(X.get_shape()[1]), 1
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    linear = tf.add(tf.matmul(X, w), b, name="linear")
    return tf.maximum(linear, 0, name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")
summary_writer = tf.summary.FileWriter("logs/relu1", tf.get_default_graph())

In [34]:
# name_scope 를 이용해 그래프를 깔끔하게 할수 있음
# unique 한 이름으로 relu_1, relu_2 로 추가하여 그래프를 작성해줌

tf.reset_default_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = int(X.get_shape()[1]), 1
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        linear = tf.add(tf.matmul(X, w), b, name="linear")
        return tf.maximum(linear, 0, name="max")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

summary_writer = tf.summary.FileWriter("logs/relu2", tf.get_default_graph())

In [35]:
summary_writer.close()

# Sharing Variables

In [41]:
# 여러 그래프 간에 공유 변수 만들기
tf.reset_default_graph()

# 함수를 이용
def relu(X, threshold):
    with tf.name_scope("relu"):
        w_shape = int(X.get_shape()[1]), 1
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        linear = tf.add(tf.matmul(X, w), b, name="linear")
        return tf.maximum(linear, threshold, name="max")

# 공유 변수 임계치 값 
threshold = tf.Variable(0.0, name="threshold")
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]
output = tf.add_n(relus, name="output")

In [42]:
tf.reset_default_graph()
# 공유 변수 사용하기 

def relu(X):
    with tf.name_scope("relu"):
        # 여러 곳에서 사용할 수 있기 때문에 한번 정의 되어 있으면 생성하지 않음
        if not hasattr(relu, "threshold"):
            relu.threshold = tf.Variable(0.0, name="threshold")
        w_shape = int(X.get_shape()[1]), 1
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        linear = tf.add(tf.matmul(X, w), b, name="linear")
        return tf.maximum(linear, relu.threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [43]:
tf.reset_default_graph()

def relu(X):
    with tf.variable_scope("relu", reuse=True):
        # get_variable() 를 이용해서 공유 변수 만들기 
        # relu/threshold 이름으로 만들어짐
        # shape 이 없기 떄문에 scala 값으로 만들어짐
        threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
        w_shape = int(X.get_shape()[1]), 1
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        linear = tf.add(tf.matmul(X, w), b, name="linear")
        return tf.maximum(linear, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

summary_writer = tf.summary.FileWriter("logs/relu6", tf.get_default_graph())
summary_writer.close()

In [44]:
tf.reset_default_graph()

def relu(X):
    with tf.variable_scope("relu"):
        threshold = tf.get_variable("threshold", shape=(), initializer=tf.constant_initializer(0.0))
        w_shape = int(X.get_shape()[1]), 1
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        linear = tf.add(tf.matmul(X, w), b, name="linear")
        return tf.maximum(linear, threshold, name="max")

# 공유 변수 만들고 사용하기 
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("", default_name="") as scope:
    first_relu = relu(X)     # create the shared variable
    scope.reuse_variables()  # then reuse it
    relus = [first_relu] + [relu(X) for i in range(4)]
output = tf.add_n(relus, name="output")

summary_writer = tf.summary.FileWriter("logs/relu8", tf.get_default_graph())
summary_writer.close()

In [45]:
# 이건 뭔지 모르겠슴...?????
tf.reset_default_graph()

with tf.variable_scope("param"):
    x = tf.get_variable("x", shape=(), initializer=tf.constant_initializer(0.))
    #x = tf.Variable(0., name="x")
with tf.variable_scope("param", reuse=True):
    y = tf.get_variable("x")

with tf.variable_scope("", default_name="", reuse=True):
    z = tf.get_variable("param/x", shape=(), initializer=tf.constant_initializer(0.))

print(x is y)
print(x.op.name)
print(y.op.name)
print(z.op.name)

True
param/x
param/x
param/x
